In [ ]:
!pip install venn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for venn: filename=venn-0.1.3-py3-none-any.whl size=19713 sha256=7c35ccc606992edaad5da7db4b120fefb635a81b87e28cff5d52d850c1ffc342
  Stored in directory: /root/.cache/pip/wheels/24/25/51/e88b9ad98643aa2c27f01a9bf5ccd10b7239e7535135d3feb1
Successfully built venn


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact 
import os 
import pandas as pd
import math
from google.colab import drive
import random
from venn import venn


drive.mount('/content/drive')

#para generar elecciones segun cantidad de metodos 
def cuales(n):
  if n==1:
    cuales=np.array(["edger"])
  if n==2:
    cuales=np.array(["edger","deseq"])
  if n==3:
    cuales=np.array(["edger","deseq","limma"])        
  if n==4:
    cuales=np.array(["edger","deseq","limma","noiseq"])
  return cuales
  
#accede a los documentos
def dataframes(metodos,rutas):
  dfs=[]
  indice=0
  for metodo in metodos:
    dfs.append(pd.read_csv(rutas[indice], sep='[;,:\s+]'))
    indice=indice+1
  return dfs    

#eliminar info no usada en cada df

def elimina (metodos,dfs,cabecera_pval,cabecera_lfc):
  indice=0
  for metodo in metodos:
    cabeceras=dfs[indice].columns.values
    eliminar=[]
    for cabecera in cabeceras:
      if cabecera != cabecera_pval[metodo] and cabecera != cabecera_lfc[metodo] and cabecera != 'ID':
        eliminar.append(cabecera)

    dfs[indice]=dfs[indice].drop(eliminar, axis=1)    
    indice=indice+1  
  
  return dfs 


#rutas de los archivos 
rutas=np.array(["/content/drive/MyDrive/archivos/Nod1_CE3vsNod61_IAC8_edgeR.txt",
                "/content/drive/MyDrive/archivos/Nod1_CE3vsNod61_IAC8_DESeq2.txt",
                "/content/drive/MyDrive/archivos/Nod1_CE3vsNod61_IAC8_limma.txt",
                "/content/drive/MyDrive/archivos/Nod1_CE3vsNod61_IAC8_NOISeq.txt"
                ])

#diccionario de cabeceras 
cabecera_pval={'edger':'FDR','deseq':'padj','limma':'adj.P.Val','noiseq':'lminusProb'}

cabecera_lfc={'edger':'logFC','deseq':'log2FoldChange','limma':'logFC','noiseq':'log2FC'}


#genera una seleccion de metodos con una cantidad de metodos aleatoria
n=random.randint(1,4)
metodos=cuales(n)

#obtenemos lista de dataframes de los metodos
dfs=dataframes(metodos,rutas)

#eliminamos columnas que no ocupamos de cada archivo
#solo tenemos p-val y lfc
dfs=elimina (metodos,dfs,cabecera_pval,cabecera_lfc)


#filtra dado un p_value
def filtra_grafica_interseccion(p_value=0.01,LFC=2.0):

  indice=0
  identificadores_dict={}
  filtro=[]
  identificadores= pd.DataFrame()
  for metodo in metodos:
    nombre_colpv=cabecera_pval[metodo]
    nombre_collfc=cabecera_lfc[metodo]
    lista=dfs[indice].index[(dfs[indice][nombre_colpv] <= p_value)&((dfs[indice][nombre_collfc]>=LFC)|(dfs[indice][nombre_collfc]<=-LFC))].tolist() 
    identificador=[]
    for elemento in lista:
      identificador.append(dfs[indice]["ID"][elemento])     
    identificadores[metodo]= pd.Series(identificador,dtype=str)   
    indice=indice+1
    identificadores_dict[metodo]=set(x for x in identificadores[metodo] if pd.isnull(x)==False)        
  venn(identificadores_dict, fmt="{size}\n({percentage:.1f}%)", fontsize=8, legend_loc="upper left")
  
  conta=0
  intersec={}
  for metodo in metodos:
    if conta==0:
      intersec=set(identificadores[metodo]).copy()
    else:
      intersec=intersec &  set(identificadores[metodo])  
    conta=conta+1  
  print()
  if len(intersec) != 0 :
    if 'edger' in metodos:
      indice_metodo=np.where(metodos == 'edger')[0][0]
      info=dfs[indice_metodo]
      clave_pv=cabecera_pval['edger']
      clave_lfc=cabecera_lfc['edger']        
    elif 'deseq' in metodos:
      indice_metodo=np.where(metodos == 'deseq')[0][0]
      info=dfs[indice_metodo]
      clave_pv=cabecera_pval['deseq']
      clave_lfc=cabecera_lfc['deseq'] 
    elif 'limma' in metodos:
      indice_metodo=np.where(metodos == 'lima')[0][0]
      info=dfs[indice_metodo]
      clave_pv=cabecera_pval['lima']
      clave_lfc=cabecera_lfc['lima'] 
    else:
      indice_metodo=np.where(metodos == 'noiseq')[0][0]
      info=dfs[indice_metodo]
      clave_pv=cabecera_pval['noiseq']
      clave_lfc=cabecera_lfc['noiseq'] 
      
    y=[]
    x=[]
    for elemento in intersec:
      consulta=info['ID']==elemento
      y.append(float(info[consulta][clave_pv]))
      x.append(float(info[consulta][clave_lfc]))
    y=pd.DataFrame(y)
    x=pd.DataFrame(x)
    y=y[0]
    x=x[0]
    y=y.apply(math.log10)
    y=-1*y
    xmin=int(x.min())
    xmax=int(x.max())
    ymax=int(y.max())
    def graf_volcano(LFC=LFC,p_value=p_value,xmin=xmin,xmax=xmax,ymax=ymax):
      p_value=-1*math.log10(p_value) #aqui pues se tranforma y con la misma funcion
      plt.figure(figsize=(10,10))
      plt.axvline(x=LFC, color="blue" )
      plt.axvline(x=-1*LFC, color="blue")
      plt.axhline(y=p_value,color="orange")
      plt.xlabel("LFC")
      plt.ylabel("-log10 FDR")
      plt.xlim(xmin,xmax)
      plt.ylim(0,ymax)
      plt.xticks(range(xmin,xmax,2))
      plt.yticks(range(-5,ymax,5))
      codigo=[]
      coordenadas=zip(x,y)
      for coordenada in coordenadas:
        if coordenada[0] <=0 and coordenada[0]<= -1*LFC and coordenada[1]>=0 and coordenada[1]>=p_value:
          codigo.append(0) #rojo
        elif coordenada[0]>=0 and coordenada[0]>= LFC and coordenada[1]>=0 and coordenada[1]>=p_value:
          codigo.append(1) #verde
        else:
          codigo.append(2) #negro
       
      codigo=np.array(codigo)
                  #rojo     verde     negro
      colores = ["#ff0000","#00cc44","#000000"]
      pinta=np.take(colores,codigo)
      plt.scatter(x, y, c=pinta)
      plt.text(xmax+2,ymax, 'down',fontsize=12,color='black', bbox={'facecolor':'red'})
      plt.text(xmax+2,ymax-10, 'up',fontsize=12,color='black', bbox={'facecolor':'green'})
      plt.text(xmax+2,ymax-25, 'other',fontsize=12,color='white', bbox={'facecolor':'black'})
      plt.text(xmax+2,ymax-40, 'p-value',fontsize=12,color='black', bbox={'facecolor':'orange'})
      plt.text(xmax+2,ymax-50, 'logFC',fontsize=12,color='white', bbox={'facecolor':'blue'})
      plt.show()
    interact(graf_volcano,p_value=(0.01,1,0.01),LFC=(0,xmax,0.1))
    None
  else:
    print("no hay interseccion para calcular volcano") 
 

    

            
              
if 'edger' in metodos:
  indice_metodo=np.where(metodos == 'edger')[0][0]
  xmax=int(dfs[indice_metodo][cabecera_lfc['edger']].max() )      
elif 'deseq' in metodos:
  indice_metodo=np.where(metodos == 'deseq')[0][0]
  xmax=int(dfs[indice_metodo][cabecera_lfc['deseq']].max() )
elif 'limma' in metodos:
  indice_metodo=np.where(metodos == 'limma')[0][0]
  xmax=int(dfs[indice_metodo][cabecera_lfc['limma']].max() )
else:
  indice_metodo=np.where(metodos == 'noiseq')[0][0]
  xmax=int(dfs[indice_metodo][cabecera_lfc['noiseq']].max() )
      
if(metodos.size !=1):
  interact(filtra_grafica_interseccion,p_value=(0.01,1,0.01),LFC=(0,xmax,0.5))
  None
else:
  y=dfs[0][cabecera_pval[metodos[0]]]
  x=dfs[0][cabecera_lfc[metodos[0]]]
  y=y.apply(math.log10)
  y=-1*y
  xmin=int(x.min())
  xmax=int(x.max())
  ymax=int(y.max())
  def volcano(LFC=2,p_value=0.01,xmin=xmin,xmax=xmax,ymax=ymax):
    p_value=-1*math.log10(p_value) #aqui pues se tranforma y con la misma funcion
    plt.figure(figsize=(10,10))
    plt.axvline(x=LFC, color="blue" )
    plt.axvline(x=-1*LFC, color="blue")
    plt.axhline(y=p_value,color="orange")
    plt.xlabel("LFC")
    plt.ylabel("-log10 FDR")
    plt.xlim(xmin,xmax)
    plt.ylim(0,ymax)
    plt.xticks(range(xmin,xmax,2))
    plt.yticks(range(-5,ymax,5))
    codigo=[]
    coordenadas=zip(x,y)
    for coordenada in coordenadas:
      if coordenada[0] <=0 and coordenada[0]<= -1*LFC and coordenada[1]>=0 and coordenada[1]>=p_value:
        codigo.append(0) #rojo
      elif coordenada[0]>=0 and coordenada[0]>= LFC and coordenada[1]>=0 and coordenada[1]>=p_value:
        codigo.append(1) #verde
      else:
        codigo.append(2) #negro
       
    codigo=np.array(codigo)
                  #rojo     verde     negro
    colores = ["#ff0000","#00cc44","#000000"]
    pinta=np.take(colores,codigo)
    plt.scatter(x, y, c=pinta)
    plt.text(xmax+2,ymax, 'down',fontsize=12,color='black', bbox={'facecolor':'red'})
    plt.text(xmax+2,ymax-10, 'up',fontsize=12,color='black', bbox={'facecolor':'green'})
    plt.text(xmax+2,ymax-25, 'other',fontsize=12,color='white', bbox={'facecolor':'black'})
    plt.text(xmax+2,ymax-40, 'p-value',fontsize=12,color='black', bbox={'facecolor':'orange'})
    plt.text(xmax+2,ymax-50, 'logFC',fontsize=12,color='white', bbox={'facecolor':'blue'})
    plt.show()
  interact(volcano,p_value=(0.01,1,0.01),LFC=(0,xmax,0.1))
  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


interactive(children=(FloatSlider(value=0.01, description='p_value', max=1.0, min=0.01, step=0.01), FloatSlide…